In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
import sys
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.stats import rankdata
import pickle
# import texthero

/kaggle/input/jigsaw-toxic-severity-rating/sample_submission.csv
/kaggle/input/jigsaw-toxic-severity-rating/validation_data.csv
/kaggle/input/jigsaw-toxic-severity-rating/comments_to_score.csv
/kaggle/input/ruddit-jigsaw-dataset/LICENSE
/kaggle/input/ruddit-jigsaw-dataset/README.md
/kaggle/input/ruddit-jigsaw-dataset/requirements.txt
/kaggle/input/ruddit-jigsaw-dataset/ruddit-comment-extraction.ipynb
/kaggle/input/ruddit-jigsaw-dataset/Dataset/create_dataset_variants.py
/kaggle/input/ruddit-jigsaw-dataset/Dataset/identityterms_group.txt
/kaggle/input/ruddit-jigsaw-dataset/Dataset/Ruddit.csv
/kaggle/input/ruddit-jigsaw-dataset/Dataset/ReadMe.md
/kaggle/input/ruddit-jigsaw-dataset/Dataset/Ruddit_individual_annotations.csv
/kaggle/input/ruddit-jigsaw-dataset/Dataset/ruddit_with_text.csv
/kaggle/input/ruddit-jigsaw-dataset/Dataset/node_dictionary.npy
/kaggle/input/ruddit-jigsaw-dataset/Dataset/post_with_issues.csv
/kaggle/input/ruddit-jigsaw-dataset/Dataset/Thread_structure.txt
/kaggle/inp

# 2/5 KAGGLE_INFERENCE=Trueだと前処理STEP1・STEP2を飛ばしpseudolabelingから開始(時短＆メモリ節約)

In [2]:
KAGGLE_INFERENCE = True
if KAGGLE_INFERENCE:
    PREPROC_STEP1 = False
    PREPROC_STEP2 = False
    TEST_INFERENCE = True
    VAL_CHECK = True
else:
    PREPROC_STEP1 = True  # 前処理STEP1を実行する場合はTrue、skipしてpickleから読む場合はFalse
    PREPROC_STEP2 = True  # 前処理STEP2を実行する場合はTrue、skipしてpickleから読む場合はFalse
    TEST_INFERENCE = True  # テストデータを使った推論時はTrue
    VAL_CHECK = True  # バリデーションチェック時はTrue

In [3]:
# PARAM_TFIDF = "char"
PARAM_TFIDF = "char_wb"
# PARAM_TFIDF = "word"
PARAM_RIDGE = 0.5

In [4]:
# 2/6 最終版
from tqdm.auto import tqdm
from bs4 import BeautifulSoup
import re 

def text_cleaning(text):
    '''
    Cleans text into a basic form for NLP. Operations include the following:-
    1. Remove special charecters like &, #, etc
    2. Removes extra spaces
    3. Removes embedded URL links
    4. Removes HTML tags
    5. Removes emojis
    
    text - Text piece to be cleaned.
    '''
    template = re.compile(r'https?://\S+|www\.\S+') #Removes website links
    text = template.sub(r'', text)
    
    soup = BeautifulSoup(text, 'lxml') #Removes HTML tags
    only_text = soup.get_text()
    text = only_text
    
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               "]+", flags=re.UNICODE)
    text = emoji_pattern.sub(r'', text)
    
    text = re.sub(r"[^a-zA-Z\d]", " ", text) #Remove special Charecters
    text = re.sub(' +', ' ', text) #Remove Extra Spaces
    text = text.strip() # remove spaces at the beginning and at the end of string

    return text

def text_cleaning_hitomoji(text):
    """1文字だけのテキストを結合する"""
    text = re.sub('\s(.)\s', r' #\1# ', text, 5)
    text = re.sub('^(.)\s', r' #\1# ', text, 5)
    text1 = re.sub('\s(.)$', r' #\1# ', text, 5)
    text2 = text1
    text_old = ""
    while text2 != text_old:
        text_old = text2
        text2 = re.sub('#\s(.)\s', r'\1# ', text_old)
    text3 = re.sub('#\s#', r'',text2)
    text4 = re.sub('#', r'',text3)
    return text4

In [5]:
# 2/6 最終版
import nltk
from nltk.corpus import stopwords
stop = stopwords.words('english') + ['utc', 'jan', 'feb', 'mar', 'apr', 'may', 'jun', 'jul', 'aug', 'sep', 'oct', 'nov', 'dec']
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()

def lemmatize_text(text):
    return [lemmatizer.lemmatize(w) for w in text]

def clean(data, col):

    # 2022/1/24 まず小文字にする
    data[col] = data[col].str.lower()

    # 2022/1/22 HTMLタグの削除など
    data[col] = data[col].map(text_cleaning)

    # 2022/1/31 追加 https://www.kaggle.com/sklasfeld/jigsaw-naive-bayes-and-ridge-regression-explained
    data[col] = data[col].str.replace("#ofc", " of fuckin course ")
    data[col] = data[col].str.replace("fggt", " faggot ")
    data[col] = data[col].str.replace("your", " your ")
    data[col] = data[col].str.replace("self", " self ")
    data[col] = data[col].str.replace("cuntbag", " cunt bag ")
    data[col] = data[col].str.replace("fartchina", " fart china ")    
    data[col] = data[col].str.replace("youi", " you i ")
    data[col] = data[col].str.replace("cunti", " cunt i ")
    data[col] = data[col].str.replace("sucki", " suck i ")
    data[col] = data[col].str.replace("pagedelete", " page delete ")
    data[col] = data[col].str.replace("cuntsi", " cuntsi ")
    data[col] = data[col].str.replace("i'm", " i am ")
    data[col] = data[col].str.replace("offuck", " of fuck ")
    data[col] = data[col].str.replace("centraliststupid", " central ist stupid ")
    data[col] = data[col].str.replace("hitleri", " hitler i ")
    data[col] = data[col].str.replace("i've", " i have ")
    data[col] = data[col].str.replace("i'll", " sick ")
    data[col] = data[col].str.replace("fuck", " fuck ")
    data[col] = data[col].str.replace("f u c k", " fuck ")
    data[col] = data[col].str.replace("shit", " shit ")
    data[col] = data[col].str.replace("bunksteve", " bunk steve ")
#     data[col] = data[col].str.replace('wikipedia', ' social medium ')
    data[col] = data[col].str.replace("faggot", " faggot ")
    data[col] = data[col].str.replace("delanoy", " delanoy ")
    data[col] = data[col].str.replace("jewish", " jewish ")
    data[col] = data[col].str.replace("sexsex", " sex ")
    data[col] = data[col].str.replace("allii", " all ii ")
    data[col] = data[col].str.replace("i'd", " i had ")
    data[col] = data[col].str.replace("'s", " is ")
    data[col] = data[col].str.replace("youbollocks", " you bollocks ")
    data[col] = data[col].str.replace("dick", " dick ")
    data[col] = data[col].str.replace("cuntsi", " cuntsi ")
    data[col] = data[col].str.replace("mothjer", " mother ")
    data[col] = data[col].str.replace("cuntfranks", " cunt ")
    data[col] = data[col].str.replace("ullmann", " jewish ")
    data[col] = data[col].str.replace("mr.", " mister ", regex=False)
    data[col] = data[col].str.replace("aidsaids", " aids ")
    data[col] = data[col].str.replace("njgw", " nigger ")
#     data[col] = data[col].str.replace("wiki", " social medium ")
    data[col] = data[col].str.replace("administrator", " admin ")
    data[col] = data[col].str.replace("gamaliel", " jewish ")
    data[col] = data[col].str.replace("rvv", " vanadalism ")
    data[col] = data[col].str.replace("admins", " admin ")
    data[col] = data[col].str.replace("pensnsnniensnsn", " penis ")
    data[col] = data[col].str.replace("pneis", " penis ")
    data[col] = data[col].str.replace("pennnis", " penis ")
    data[col] = data[col].str.replace("pov.", " point of view ", regex=False)
    data[col] = data[col].str.replace("vandalising", " vandalism ")
    data[col] = data[col].str.replace("cock", " dick ")
    data[col] = data[col].str.replace("asshole", " asshole ")
    data[col] = data[col].str.replace("youi", " you ")
    data[col] = data[col].str.replace("afd", " all fucking day ")
    data[col] = data[col].str.replace("sockpuppets", " sockpuppetry ")
    data[col] = data[col].str.replace("iiprick", " iprick ")
    data[col] = data[col].str.replace("penisi", " penis ")
    data[col] = data[col].str.replace("warrior", " warrior ")
    data[col] = data[col].str.replace("loil", " laughing out insanely loud ")
    data[col] = data[col].str.replace("vandalise", " vanadalism ")
    data[col] = data[col].str.replace("helli", " helli ")
    data[col] = data[col].str.replace("lunchablesi", " lunchablesi ")
    data[col] = data[col].str.replace("special", " special ")
    data[col] = data[col].str.replace("ilol", " i lol ")
    data[col] = data[col].str.replace(r'\b[uU]\b', 'you', regex=True)
    data[col] = data[col].str.replace(r"what's", "what is ", regex=False)
    data[col] = data[col].str.replace(r"\'s", " is ", regex=False)
    data[col] = data[col].str.replace(r"\'ve", " have ", regex=False)
    data[col] = data[col].str.replace(r"can't", "cannot ", regex=False)
    data[col] = data[col].str.replace(r"n't", " not ", regex=False)
    data[col] = data[col].str.replace(r"i'm", "i am ", regex=False)
    data[col] = data[col].str.replace(r"\'re", " are ", regex=False)
    data[col] = data[col].str.replace(r"\'d", " would ", regex=False)
    data[col] = data[col].str.replace(r"\'ll", " will ", regex=False)
    data[col] = data[col].str.replace(r"\'scuse", " excuse ", regex=False)    
#     data[col] = data[col].str.replace(r"what's", "what is ", regex=True)    
#     data[col] = data[col].str.replace(r"\'ve", " have ", regex=True)
#     data[col] = data[col].str.replace(r"can't", "cannot ", regex=True)
#     data[col] = data[col].str.replace(r"n't", " not ", regex=True)
#     data[col] = data[col].str.replace(r"i'm", "i am ", regex=True)
#     data[col] = data[col].str.replace(r"\'re", " are ", regex=True)
#     data[col] = data[col].str.replace(r"\'d", " would ", regex=True)
#     data[col] = data[col].str.replace(r"\'ll", " will ", regex=True)
#     data[col] = data[col].str.replace(r"\'scuse", " excuse ", regex=True)
    data[col] = data[col].str.replace(r"\'s", " ", regex=False)

    # 2022/1/22 数字は削除する
    data[col] = data[col].str.replace(r"[\d]", " ", regex=True)

    # Clean some punctutations
    data[col] = data[col].str.replace('\n', ' \n ')
    data[col] = data[col].str.replace(r'([a-zA-Z]+)([/!?.])([a-zA-Z]+)',r'\1 \2 \3', regex=True)
    # Replace repeating characters more than 3 times to length of 3
    data[col] = data[col].str.replace(r'([*!?\'])\1\1{2,}',r'\1\1\1', regex=True)
    # Add space around repeating characters
    data[col] = data[col].str.replace(r'([*!?\']+)',r' \1 ', regex=True)    
    # patterns with repeating characters 
    data[col] = data[col].str.replace(r'([a-zA-Z])\1{2,}\b',r'\1\1', regex=True)
    data[col] = data[col].str.replace(r'([a-zA-Z])\1\1{2,}\B',r'\1\1\1', regex=True)
    data[col] = data[col].str.replace(r'[ ]{2,}',' ', regex=True).str.strip()   
    data[col] = data[col].str.replace(r'[ ]{2,}',' ', regex=True).str.strip()   

    # 2022/1/22 特殊文字を除去する
    # 2022/1/26 特殊文字を「_」に変換する
    data[col] = data[col].str.replace(r'[^a-zA-Z\d]', r' ', regex=True)

    # 2022/1/22 完全に空白を除去する
    if PARAM_TFIDF == "char":
        data[col] = data[col].map(lambda x: '_'.join([word for word in x.split() if word not in (stop)]))
    #     data[col] = data[col].apply(lambda x: ''.join([word for word in x.split() if word not in (stop)]))
    else:
        data[col] = data[col].map(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
    
    data[col] = data[col].map(text_cleaning_hitomoji)

#     # 2022/1/22 レンマタイズ（活用を除去）→精度あがらず
#     data[col] = data[col].map(lambda x: ' '.join([lemmatizer.lemmatize(w) for w in w_tokenizer.tokenize(x)]))

#     # 2022/1/29 stopワードを除去（追加分）→これも精度あがらず
#     data[col] = data[col].map(lambda x: ' '.join([word for word in x.split() if word not in (stop_tsuika)]))
    
    return data

In [6]:
import scipy
# ベクトルとモデルを生成
def read_and_clean(path, list_col_name):
    df = pd.read_csv(path)
    for col in list_col_name:
        df = clean(df, col)
    print(df.head(3))
    return df

def get_vec_model(sr_all_text, df, df_pseudo, list_loop, param_bootstrap):
    """vecは1つ、modelは複数返す"""
    list_model = []
    #     count_min = int((len(sr_all_text) / 10000) ** 0.5) + 1
    vec = TfidfVectorizer(analyzer=PARAM_TFIDF, max_df=0.9, min_df=1, ngram_range=(2, 6))
    vec.fit(sr_all_text)
    if param_bootstrap >= 1:
        for seed_now in list_loop:
            # 有害なものと同件の無害なものをブートストラップする　ここから
            # ①有害なものの件数
            min_len = len(sr_all_text) * param_bootstrap
            str_undersample = f"アンダーサンプリング有害コメントの{param_bootstrap}倍"
            # ②無害なものを抽出
            if min_len < len(df['y'] == 0):
                df_y0_bootstrap = df[df['y'] == 0].sample(n=min_len, random_state=seed_now)
                print(str_undersample)
            else:
                df_y0_bootstrap = df[df['y'] == 0]
                print("アンダーサンプリング全件")
            # ③新しいデータフレームを作る（スコアは正規化する）
            df_now = pd.concat([df[df['y'] > 0], df_y0_bootstrap])
            df_now["y"] = scipy.stats.zscore(df_now["y"])
            # 有害なものと同件の無害なものをブートストラップする　ここまで
            print(df_now.shape)
            df_now = pd.concat([df_now, df_pseudo]).reset_index(drop=True)
            X = vec.transform(df_now['text'])
            y = np.around(df_now["y"], decimals=2)
            model = Ridge(alpha=PARAM_RIDGE)
            model.fit(X, y)
            list_model.append(model)
    else:
        # スコアを正規化する
        df["y"] = scipy.stats.zscore(df["y"])
        df_now = pd.concat([df, df_pseudo]).reset_index(drop=True)
        X = vec.transform(df_now['text'])
        y = np.around(df_now["y"], decimals=2)
        for ridge_param_now in list_loop:
            print(df_now.shape)
            model = Ridge(alpha=ridge_param_now)
            model.fit(X, y)
            list_model.append(model)
        
    return vec, list_model

def get_test(vec, list_model, df):
    X = vec.transform(df['text'])
    preds = np.zeros(X.shape[0])
    for model in list_model:
        preds += model.predict(X)
    # 2/6 バグ修正
    preds /= len(list_model)
    return preds

def get_omomi(df, omomi, cname):
    df[cname] = 0
    for k, v in omomi.items():
        if k in df.columns:
            df[cname] += df[k] * v
    return df

# 前処理STEP1

In [7]:
if PREPROC_STEP1:
    df_test = read_and_clean("../input/jigsaw-toxic-severity-rating/comments_to_score.csv", ['text'])
    df_test.to_pickle("df_test_0206.pkl")
    df_val = read_and_clean("../input/jigsaw-toxic-severity-rating/validation_data.csv", ['less_toxic', 'more_toxic'])
    df_val.to_pickle("df_val_0206.pkl")
    df_train = read_and_clean("../input/jigsaw-toxic-comment-classification-challenge/train.csv", ['comment_text'])
    df_train.to_pickle("df_train_0206.pkl")
    df_rud =read_and_clean("../input/ruddit-jigsaw-dataset/Dataset/ruddit_with_text.csv", ['txt'])
    df_rud.to_pickle("df_rud_0206.pkl")
    df_mulwi = read_and_clean('../input/jigsaw-multilingual-toxic-comment-classification/jigsaw-toxic-comment-train.csv', ['comment_text'])
    df_mulwi.to_pickle("df_mulwi_0206.pkl")
    df_uni = read_and_clean('../input/jigsaw-unintended-bias-in-toxicity-classification/train.csv', ['comment_text'])
    df_uni.to_pickle("df_uni_0206.pkl")
else:
    df_test = read_and_clean("../input/jigsaw-toxic-severity-rating/comments_to_score.csv", ['text'])
    df_val = pd.read_pickle("../input/jigsaw-202201/df_val_0206.pkl")
    df_train = pd.read_pickle("../input/jigsaw-202201/df_train_0206.pkl")
    df_rud = pd.read_pickle("../input/jigsaw-202201/df_rud_0206.pkl")
    df_mulwi = pd.read_pickle("../input/jigsaw-202201/df_mulwi_0206.pkl")
    df_uni = pd.read_pickle("../input/jigsaw-202201/df_uni_0206.pkl")

   comment_id                                               text
0      114890  gjalexei asked whether anti editorializing pol...
1      732895               looks like abuser please look thanks
2     1139051  confess complete apparently blissful ignorance...


In [8]:
# ①classification
dic_omomi = {'obscene': 0.16, 'toxic': 0.32, 'insult': 0.64, 'threat': 1.5, 'identity_hate': 1.5, 'severe_toxic': 2, 'identity_attack': 1.5, 'severe_toxicity': 2}
df_train = get_omomi(df_train, dic_omomi, 'toxic_sum')
sr_all_text1 = pd.concat([df_train[df_train['toxic_sum'] > 0]['comment_text']]).drop_duplicates()
sr_all_text1.shape

(16030,)

In [9]:
# ②ruddit
df_rud['offensiveness_score'].clip(lower=0, inplace=True)
sr_all_text2 = df_rud[df_rud['offensiveness_score'] > 0]['txt'].drop_duplicates()
sr_all_text2.shape

(2312,)

In [10]:
# ③multiのwiki
df_mulwi = get_omomi(df_mulwi, dic_omomi, 'toxic_sum')
sr_all_text3 = df_mulwi[df_mulwi['toxic_sum'] > 0]['comment_text'].drop_duplicates()
sr_all_text3.shape

(22176,)

In [11]:
# ④unintended - 1
list_normal = ['asian', 'atheist', 'bisexual',
       'black', 'buddhist', 'christian', 'female', 'heterosexual', 'hindu',
       'homosexual_gay_or_lesbian', 'intellectual_or_learning_disability',
       'jewish', 'latino', 'male', 'muslim', 'other_disability',
       'other_gender', 'other_race_or_ethnicity', 'other_religion',
       'other_sexual_orientation', 'physical_disability',
       'psychiatric_or_mental_illness', 'transgender', 'white']

In [12]:
# ④unintended - 2
df_uni = get_omomi(df_uni, dic_omomi, 'toxic_sum')
df_uni['normal_sum'] = df_uni[list_normal].sum(axis=1)
df_uni = df_uni[df_uni['normal_sum'] == 0]
df_uni_toxic = df_uni.nlargest(50000, 'toxic_sum', keep="all") # toxicな上位50000件だけを取り出す
df_uni_non_toxic = df_uni[df_uni['toxic_sum'] == 0]
df_uni = pd.concat([df_uni_toxic, df_uni_non_toxic]).reset_index(drop=True)
sr_all_text4 = df_uni[df_uni['toxic_sum'] > 0]['comment_text'].drop_duplicates()
sr_all_text4.shape

(49144,)

# 前処理STEP2：TF-IDFベクトライザ及びRidge推論器を生成

In [13]:
if PREPROC_STEP2:
    # ①classification
    print(f"df_train:{df_train.shape}")
    df = df_train[['comment_text', 'toxic_sum']].rename(columns={'comment_text': 'text', 'toxic_sum': 'y'})
    df_pseudo = pd.DataFrame()
    vec1, list_model1 = get_vec_model(sr_all_text1, df, df_pseudo, [i + 217 for i in range(10)], 5)

In [14]:
if PREPROC_STEP2:
    # ②ruddit
    print(f"df_rud:{df_rud.shape}")
    df = df_rud[['txt', 'offensiveness_score']].rename(columns={'txt': 'text', 'offensiveness_score': 'y'})
    df_pseudo = pd.DataFrame()
    vec2, list_model2 = get_vec_model(sr_all_text2, df, df_pseudo, [0.5, 1.0, 1.5, 2.0], 0)

In [15]:
if PREPROC_STEP2:
    # ③multiのwiki
    print(f"df_mulwi:{df_mulwi.shape}")
    df = df_mulwi[['comment_text', 'toxic_sum']].rename(columns={'comment_text': 'text', 'toxic_sum': 'y'})
    df_pseudo = pd.DataFrame()
    vec3, list_model3 = get_vec_model(sr_all_text3, df, df_pseudo, [i + 217 for i in range(10)], 5)

In [16]:
if PREPROC_STEP2:
    # ④unintended
    print(f"df_uni:{df_uni.shape}")
    df = df_uni[['comment_text', 'toxic_sum']].rename(columns={'comment_text': 'text', 'toxic_sum': 'y'})
    df_pseudo = pd.DataFrame()
    vec4, list_model4 = get_vec_model(sr_all_text4, df, df_pseudo, [i + 217 for i in range(10)], 5)

# 前処理STEP2：生成したTF-IDFベクトライザ及びRidge推論器をpickle化

In [17]:
if PREPROC_STEP2:
    # 生成したベクトライザとridge推論器を保存する
    list_vec = [vec1, vec2, vec3, vec4]
    with open("list_vec_0206_x5_seed10.pkl", mode='wb') as f:
        pickle.dump(list_vec, f)
    list_list_model = [list_model1, list_model2, list_model3, list_model4]
    with open("list_list_model_0206_x5_seed10.pkl", mode='wb') as f:
        pickle.dump(list_list_model, f)
else:
    # あらかじめ作っておいたベクトライザとridge推論器をdatasetから読み込む
    with open("../input/jigsaw-202201/list_vec_0206_x5_seed10.pkl", mode='rb') as f:
        vec1, vec2, vec3, vec4 = pickle.load(f)
    with open("../input/jigsaw-202201/list_list_model_0206_x5_seed10.pkl", mode='rb') as f:
        list_model1, list_model2, list_model3, list_model4 = pickle.load(f)

# ここから本処理：pseudoラベリング用に事前推論する（m1234_pseudo_score）

In [18]:
if TEST_INFERENCE:
    df_test['m1score'] = get_test(vec1, list_model1, df_test)
    df_test['m2score'] = get_test(vec2, list_model2, df_test)
    df_test['m3score'] = get_test(vec3, list_model3, df_test)
    df_test['m4score'] = get_test(vec4, list_model4, df_test)

In [19]:
# pseudo labeling（正規化しているのでスケールを気にせず単純平均でよい）
df_pseudo = df_test.copy()
df_pseudo['m1234_pseudo_score'] = df_pseudo[['m1score', 'm2score', 'm3score', 'm4score']].mean(axis=1)
print(df_pseudo)

      comment_id                                               text   m1score  \
0         114890  gjalexei asked whether anti editorializing pol... -0.457048   
1         732895               looks like abuser please look thanks  0.312879   
2        1139051  confess complete apparently blissful ignorance...  0.022502   
3        1434512  freud ideas certainly much discussed today wou... -0.308689   
4        2084821  laundry list stupid allegations scooped god kn...  0.046197   
...          ...                                                ...       ...   
7532   504235362                            go away annoying vandal  0.152712   
7533   504235566                                        user vandal -0.082836   
7534   504308177  sorry sound like pain one following tad stalki... -0.530685   
7535   504570375     well pretty fuck ing irrelevant unblocked aint  0.504162   
7536   504598250  team name great britain northern ireland blind...  0.059553   

       m2score   m3score   

# 再び、TF-IDFベクトライザ及びRidge推論器を生成

In [20]:
# ①classification
print(f"df_train:{df_train.shape}")
df = df_train[['comment_text', 'toxic_sum']].rename(columns={'comment_text': 'text', 'toxic_sum': 'y'})
df_pseudo_add = df_pseudo.rename(columns={'m1234_pseudo_score': 'y'})
sr_all_text11 = pd.concat([df[df['y'] > 0]['text'], df_pseudo_add[df_pseudo_add['y'] > 0]['text']]).drop_duplicates()
print(sr_all_text11.shape)
vec11, list_model11 = get_vec_model(sr_all_text11, df, df_pseudo_add, [i + 217 for i in range(5)], 5)

df_train:(159571, 9)
(18039,)
アンダーサンプリング有害コメントの5倍
(106420, 2)
アンダーサンプリング有害コメントの5倍
(106420, 2)
アンダーサンプリング有害コメントの5倍
(106420, 2)
アンダーサンプリング有害コメントの5倍
(106420, 2)
アンダーサンプリング有害コメントの5倍
(106420, 2)


In [21]:
# ②ruddit
print(f"df_rud:{df_rud.shape}")
df = df_rud[['txt', 'offensiveness_score']].rename(columns={'txt': 'text', 'offensiveness_score': 'y'})
df_pseudo_add = df_pseudo.rename(columns={'m1234_pseudo_score': 'y'})
sr_all_text12 = pd.concat([df[df['y'] > 0]['text'], df_pseudo_add[df_pseudo_add['y'] > 0]['text']]).drop_duplicates()
print(sr_all_text12.shape)
vec12, list_model12 = get_vec_model(sr_all_text12, df, df_pseudo_add, [0.5, 1.0, 1.5, 2.0], 0)

df_rud:(5838, 5)
(7678,)
(13375, 7)
(13375, 7)
(13375, 7)
(13375, 7)


In [22]:
# ③multiのwiki
print(f"df_mulwi:{df_mulwi.shape}")
df = df_mulwi[['comment_text', 'toxic_sum']].rename(columns={'comment_text': 'text', 'toxic_sum': 'y'})
df_pseudo_add = df_pseudo.rename(columns={'m1234_pseudo_score': 'y'})
sr_all_text13 = pd.concat([df[df['y'] > 0]['text'], df_pseudo_add[df_pseudo_add['y'] > 0]['text']]).drop_duplicates()
print(sr_all_text13.shape)
vec13, list_model13 = get_vec_model(sr_all_text13, df, df_pseudo_add, [i + 217 for i in range(5)], 5)

df_mulwi:(223549, 9)
(24185,)
アンダーサンプリング有害コメントの5倍
(143393, 2)
アンダーサンプリング有害コメントの5倍
(143393, 2)
アンダーサンプリング有害コメントの5倍
(143393, 2)
アンダーサンプリング有害コメントの5倍
(143393, 2)
アンダーサンプリング有害コメントの5倍
(143393, 2)


In [23]:
# ④unintended
print(f"df_uni:{df_uni.shape}")
df = df_uni[['comment_text', 'toxic_sum']].rename(columns={'comment_text': 'text', 'toxic_sum': 'y'})
df_pseudo_add = df_pseudo.rename(columns={'m1234_pseudo_score': 'y'})
sr_all_text14 = pd.concat([df[df['y'] > 0]['text'], df_pseudo_add[df_pseudo_add['y'] > 0]['text']]).drop_duplicates()
print(sr_all_text14.shape)
vec14, list_model14 = get_vec_model(sr_all_text14, df, df_pseudo_add, [i + 217 for i in range(3)], 5)

df_uni:(1207439, 47)
(54506,)
アンダーサンプリング有害コメントの5倍
(322530, 2)
アンダーサンプリング有害コメントの5倍
(322530, 2)
アンダーサンプリング有害コメントの5倍
(322530, 2)


In [24]:
if TEST_INFERENCE:
    df_test['m11score'] = get_test(vec11, list_model11, df_test)
    df_test['m12score'] = get_test(vec12, list_model12, df_test)
    df_test['m13score'] = get_test(vec13, list_model13, df_test)
    df_test['m14score'] = get_test(vec14, list_model14, df_test)

# バリデーションチェック

In [25]:
def val_check(vec, list_model, df, cname1, cname2):
    X_less_toxic = vec.transform(df['less_toxic'])
    X_more_toxic = vec.transform(df['more_toxic'])
    df[cname1] = 0
    df[cname2] = 0
    for model in list_model:    
        df['less'] = model.predict(X_less_toxic)
        df['more'] = model.predict(X_more_toxic)
        df_lessmore = pd.DataFrame(scipy.stats.zscore(df[['less', 'more']].to_numpy(), axis=None), index=df.index, columns=['less_zscore', 'more_zscore'])
        df[cname1] += df_lessmore['less_zscore']
        df[cname2] += df_lessmore['more_zscore']
    print(cname1, cname2)

    print((df[cname1] < df[cname2]).mean())
    return df

In [26]:
# list_model_tmp = [list_model1[0]]
if VAL_CHECK:
    df_val = val_check(vec1, list_model1, df_val, 'm1p1', 'm1p2')
    df_val = val_check(vec11, list_model11, df_val, 'm11p1', 'm11p2')
    # ノーマル0.670
    # 有害データのみ0.667
    # 1～0.8、(2, 6)にして0.669
    # v16 0.6688
    # v17 0.6700
    # v17-2 0.6702
    # v19 cleanを適用すると精度が下がる0.663
    # v19-2 HTMLcleanを適用すると精度上がる0.665
    # v21 空白除去すると精度高い0.671
    # v21-2 記号除去すると精度高い0.674
    # v21-3 char_wbに戻すと0.669
    # v21-3 小文字にしたら0.671まで改善
    # v27 単体で0.6867
    # v31 0.6867
    # v35 0.6763
    # v35-2 0.677
    # v35-2 0.677
    # v35-2 0.678
    # v35-2 0.678
    # v35-3 0.678
    # v37 0.678
    # v38 0.683
    # v39 0.683
    # v41 0.6839
    # v47 0.6847
    # v48 0.6861
    # v48-2 0.6870
    # v51 0.6773(3倍) 0.6713(pseudo) バグあり
    # v51-2 0.6773(3倍) 0.6707(psuedo) バグあり
    # v52 0.6833(5倍) 0.6795(pseudo) バグあり
    # v53 0.6841(5倍) 0.6843(pseudo)
    # v58 0.6818(5倍) 0.6776(pseudo > 0のみ)
    # v59 0.6801(5×3倍) 0.6802(pseudo5×3倍)
    # v61 0.6838(x5 seed x10) 0.6794(pseudo x5 seed x5) バグあり
    # v61 0.6838(x5 seed x10) 0.6873(pseudo x5 seed x5) バグ修正後

m1p1 m1p2
0.6838381825428458
m11p1 m11p2
0.6873588415039192


In [27]:
if VAL_CHECK:
    df_val = val_check(vec2, list_model2, df_val, 'm2p1', 'm2p2')
    df_val = val_check(vec12, list_model12, df_val, 'm12p1', 'm12p2')
    # 0.632
    # 1～0.8、(2, 6)にして0.647→LB悪化したため元に戻す
    # v16 0.6320
    # v17 0.6350
    # v17-2 0.6362
    # v19 cleanを適用すると精度が下がる0.6324
    # v19-2 HTMLcleanを適用すると精度上がる0.6335
    # v21 空白除去すると精度悪い0.627
    # v21-2 記号除去すると精度高い0.636
    # v21-3 char_wbに戻すと0.638
    # v21-3 小文字にしたら0.636に悪化
    # v27 単体で0.6199
    # v35 0.638
    # v35-2 0.638
    # v35-2 0.641
    # v35-2 0.643
    # v35-2 0.641
    # v35-3 0.641
    # v37 0.649
    # v38 0.651
    # v39 0.650
    # v41 0.6514
    # v48 0.6538
    # v51 0.6536 0.6362(pseudo) バグあり
    # v51-2 0.6536 0.6536(psuedo) バグあり
    # v52 0.6532 0.6620(pseudo) バグあり
    # v53 0.6532 0.6867(pseudo)
    # v58 0.6504 0.6776(pseudo > 0のみ)
    # v59 0.6504 0.6851(pseudo)
    # v61 0.6532 0.6852(pseudo) バグあり
    # v61 0.6532 0.6825(pseudo) バグ修正後

m2p1 m2p2
0.6532483060980471
m12p1 m12p2
0.6825760595190647


In [28]:
if VAL_CHECK:
    df_val = val_check(vec3, list_model3, df_val, 'm3p1', 'm3p2')
    df_val = val_check(vec13, list_model13, df_val, 'm13p1', 'm13p2')
    # v35 0.676
    # v35-2 0.676
    # v35-2 0.678
    # v35-2 0.678
    # v35-2 0.679
    # v35-3 0.679
    # v37 0.679
    # v38 0.684
    # v39 0.684
    # v41 0.6844
    # v48 0.6861
    # v48-2 0.6867
    # v51 0.6793(3倍) 0.6794(pseudo) バグあり
    # v51-2 0.6793(3倍) 0.6782(pseudo) バグあり
    # v52 0.6794(3倍) 0.6807(pseudo) バグあり
    # v53 0.6822(5倍) 0.6826(pseudo)
    # v58 0.6809(5倍) 0.6751(pseudo > 0のみ)
    # v59 0.6787(5×3倍) (pseudo5×3倍)
    # v61 0.6824(x5 seed x10) 0.6760(pseudo x5 seed x5) バグあり
    # v61 0.6824(x5 seed x10) 0.6847(pseudo x5 seed x5) バグ修正後

m3p1 m3p2
0.6824432044639298
m13p1 m13p2
0.6847681679287897


In [29]:
if VAL_CHECK:
    df_val = val_check(vec4, list_model4, df_val, 'm4p1', 'm4p2')
    df_val = val_check(vec14, list_model14, df_val, 'm14p1', 'm14p2')
    # v51 0.6795(2倍) 0.6764(pseudo) バグあり
    # v53 0.6710(5倍) 0.6829(pseudo)
    # v58 0.6737(5倍) 0.6653(pseudo > 0のみ)
    # v59 0.6709(3×3倍) 0.6807(pseudo3×3倍)
    # v61 0.6712(x5 seed x10) 0.6763(pseudo x5 seed x5) バグあり
    # v61 0.6712(x5 seed x10) 0.6760(pseudo x5 seed x5) バグ修正後

m4p1 m4p2
0.6712169523050352
m14p1 m14p2
0.6760661618174572


In [30]:
# 2/6 v62 submit時はテストデータが増えるので、pseudoラベリングの重みを減らした方がよさそう→pseudo50%
# 2/6 v61バグ修正後 1:5が一番CV高いので、1:5にする。（pseudo83%）
# 2/5 v58田村メモ m11～m14の重みを高めすぎるとCVが悪化するので、元モデルと4:1でマージする。
dic_omomi = {"m1": 1, "m2": 1, "m3": 1, "m4": 1, "m11": 1, "m12": 1, "m13": 1, "m14": 1}
if VAL_CHECK:
    df_val['p1'] = 0
    df_val['p2'] = 0
    for k, v in dic_omomi.items():
        print(k)
        df_val['p1'] += df_val[f'{k}p1'] * v
        df_val['p2'] += df_val[f'{k}p2'] * v
    print((df_val['p1'] < df_val['p2']).mean())
    # Validation Accuracy
    # 0.678
    # 0.6786
    # 1+2+3+5+7＝0.692
    # v16 2021/1/19 0.6892
    # v17 0.6865
    # v17-2 0.6863
    # v19 0.6882に上がった
    # 2+3+4+5+6＝0.690
    # v19-2 0.6910に上がった
    # v21 空白除去すると精度やや下がる 0.690
    # v22 記号を除去したら上がった0.6917
    # v35 0.682
    # v35-2 0.682
    # 0.683
    # v35-2 0.685
    # v35-3 0.685
    # v37 0.6916
    # v41 0.6919
    # v46 0.6956
    # v48 0.6935
    # v51 0.6822
    # v53 0.6925 10-15-5-35
    # v58 0.6902 5倍だと元モデルの精度が悪い
    # v59 0.6907 3倍だと1つ1つの精度は悪いがアンサンブルすると5倍より良くなる
    # v59 0.6899(9:1) 0.6896(4:1) 0.6883(1:1)
    # v61 0.6910(m1～4のみ)
    # v61 0.6921(重み1:5) バグ修正後
    # v61 0.6912(重み1:1) バグ修正後

m1
m2
m3
m4
m11
m12
m13
m14
0.6912780656303973


# バリデーション14251件のスコアをCSV化

In [31]:
if VAL_CHECK:
    # df_val.to_csv("df_val.csv", index=False)
    df_val_moto = pd.read_csv("../input/jigsaw-toxic-severity-rating/validation_data.csv")
    df_val_concat1 = pd.concat([df_val_moto['less_toxic'], df_val['p1']], axis=1).rename(columns={'less_toxic' : 'text', 'p1': 'score'})
    df_val_concat2 = pd.concat([df_val_moto['more_toxic'], df_val['p2']], axis=1).rename(columns={'more_toxic' : 'text', 'p2': 'score'})
    df_val_concat = pd.concat([df_val_concat1, df_val_concat2]).reset_index(drop=True)
    df_val_concat = df_val_concat.drop_duplicates().reset_index(drop=True)
    df_val_concat.to_csv("df_val14251_tamura0206v62.csv", index=False)
    print(df_val_concat)

                                                    text       score
0                This article sucks \n\nwoo woo wooooooo    2.001099
1      "And yes, people should recognize that but the...  -36.039877
2       Western Media?\n\nYup, because every crime in...  -24.241773
3      And you removed it! You numbskull! I don't car...  -12.329818
4       smelly vagina \n\nBluerasberry why don't you ...   14.996967
...                                                  ...         ...
14246   Hey you fucking jew \nwhy the fuck did you ge...  173.233618
14247   i hate your guts \n\ni hope you feel good abo...    7.305034
14248  ":::fuck you animal! remember this shit yourse...   69.134670
14249      Some Say She is a little mothyer fukin bitchy   32.490363
14250                       Piss off you slant eyed-gook   13.360951

[14251 rows x 2 columns]


# テストデータのスコア計算

In [32]:
df_test['sum_score'] = 0
for k, v in dic_omomi.items():
    print(k)
    df_test['sum_score'] += df_test[f'{k}score'] * v
df_test['score'] = rankdata(df_test['sum_score'], method='ordinal') # 同値に無理やり順位を付ける場合はordinal
df_test[['comment_id', 'score']].to_csv("submission.csv", index=False)
df_test

m1
m2
m3
m4
m11
m12
m13
m14


,comment_id,text,m1score,m2score,m3score,m4score,m11score,m12score,m13score,m14score,sum_score,score
0,114890,gjalexei asked whether anti editorializing pol...,-0.457048,0.188498,-0.176815,-0.286551,-0.405754,-0.093755,-0.177341,-0.352116,-1.760883,701
1,732895,looks like abuser please look thanks,0.312879,-0.226348,0.249732,0.368422,0.314798,-0.076618,0.255697,0.348907,1.547470,3355
2,1139051,confess complete apparently blissful ignorance...,0.022502,0.150208,0.035300,0.070720,0.052958,0.094190,0.032764,-0.038913,0.419730,2567
3,1434512,freud ideas certainly much discussed today wou...,-0.308689,0.101604,-0.375382,-0.464779,-0.273844,-0.101876,-0.350290,-0.495752,-2.269008,388
4,2084821,laundry list stupid allegations scooped god kn...,0.046197,0.317173,0.165799,0.118146,0.128581,0.343045,0.233642,0.142724,1.495305,3317
...,...,...,...,...,...,...,...,...,...,...,...,...
7532,504235362,go away annoying vandal,0.152712,-0.421490,0.134088,-0.010699,0.118355,-0.099579,0.101288,-0.005427,-0.030752,2156
7533,504235566,user vandal,-0.082836,-0.769679,-0.026129,-0.314493,-0.135856,-0.343289,-0.070324,-0.387961,-2.130567,464
7534,504308177,sorry sound like pain one following tad stalki...,-0.530685,0.124866,-0.555368,-0.369778,-0.472119,-0.201527,-0.502755,-0.445282,-2.952648,118
7535,504570375,well pretty fuck ing irrelevant unblocked aint,0.504162,1.647653,0.603563,0.718359,0.649643,1.158418,0.708011,0.715865,6.705675,5496
